## Implementing Simhash

https://github.com/leonsim/simhash  
https://github.com/seomoz/simhash-py

### Simhash and hamming distance  
https://en.wikipedia.org/wiki/SimHash  
https://en.wikipedia.org/wiki/Hamming_distance

### LSH on Spark  
https://databricks.com/blog/2017/05/09/detecting-abuse-scale-locality-sensitive-hashing-uber-engineering.html

In [1]:
import pandas as pd
import re
import json
from itertools import combinations, takewhile
import collections

from simhash import Simhash, SimhashIndex

In [2]:
text1 = "texas online preparatory school now accepting enrollment for 2018 school year"
text2 = "texas online preparatory school now accepting enrollment for 2019 school year"
text3 = "No one knows how Google Duplex will work with eavesdropping laws"
text4 = "Google sells the future, powered by your personal data"
text5 = "Google And Amazon Raise The Volume On Conversational Commerce"
text6 = "Google"
text7 = "Apple made more Profit in the last Quarter than Amazon made since its Inception"
text8 = "Recipe: Apple Pear Sour Cocktail"

all_documents = [text1,text2,text3,text4,text5,text6,text7,text8]

In [3]:
data = dict(map(lambda t: (t[0], t[1]), enumerate(all_documents)))

In [4]:
def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]

In [5]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=3) #`k` is the tolerance

print (index.bucket_size())

31


In [6]:
s1 = Simhash(get_features(u'Google sells the future, powered by your personal data'))
print (index.get_near_dups(s1))

['3']


In [7]:
index.add('4', s1)
print (index.get_near_dups(s1))

['4', '3']


In [8]:
print (Simhash("Google sells the future, powered by your personal data").\
       distance(Simhash("Google sells the future, powered by your personal data")))

0


In [9]:
print (Simhash("Google sells the future, powered by AI on personal data").\
       distance(Simhash("Google sells the future, powered by your personal data")))

13


In [10]:
print (Simhash("Apple sells the future, powered by your personal data").\
       distance(Simhash("Google sells the future, powered by your personal data")))

9


## Applying Simhash to real volume of data

In [11]:
df = spark.read.json('file:///project/msca/kadochnikov/webhose/news_autos_2018_09.json')
df.cache()
df.count()

57200

In [12]:
#df.printSchema()

In [13]:
news_df = df.select('crawled', 'title', 'language').toPandas()

In [14]:
pd.set_option('max_colwidth',100)    
news_df.head(10)
news_df.head(5)

,crawled,title,language
0,2018-08-24T00:00:33.010+03:00,2015 BMW 528I CPO *B21487* $24995,english
1,2018-08-24T00:00:56.057+03:00,2001 Mercedes-Benz ml320 $3500,english
2,2018-08-24T00:00:57.114+03:00,*2009* *Mercedes-Benz* *E350* *Base* (_Mercedes-Benz_ _E350_ _Sedan_) $11991,english
3,2018-08-24T00:00:58.046+03:00,2000 MERCEDES BENZ DESIGNO SILVER S500 (Granite bay) $4000,english
4,2018-08-24T00:00:58.046+03:00,2007 BMW 550i $9900,english


In [15]:
data = news_df['title'].to_dict()

In [16]:
data_first = {k: data[k] for k in list(data)[:5]}
data_first

{0: '2015 BMW 528I CPO *B21487* $24995',
 1: '2001 Mercedes-Benz ml320 $3500',
 2: '*2009* *Mercedes-Benz* *E350* *Base* (_Mercedes-Benz_ _E350_ _Sedan_) $11991',
 3: '2000 MERCEDES BENZ DESIGNO SILVER S500 (Granite bay) $4000',
 4: '2007 BMW 550i $9900'}

### Finding near-duplicates

In [17]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=3)

print (index.bucket_size())

109932


In [18]:
s1 = Simhash(get_features(u"Mercedes Benz"))
s1_dups = index.get_near_dups(s1)
print (index.get_near_dups(s1))

['23321', '30313', '13672', '25078', '13528', '11667', '13532', '38525', '37996']


In [19]:
news_df.iloc[s1_dups]

,crawled,title,language
23321,2018-09-05T13:25:07.016+03:00,Mercedes Benz,english
30313,2018-09-08T12:35:00.000+03:00,Mercedes Benz,english
13672,2018-08-31T08:50:46.001+03:00,mercedes Benz,english
25078,2018-09-06T05:32:58.011+03:00,mercedes-benz,english
13528,2018-08-31T07:20:15.003+03:00,mercedes Benz,english
11667,2018-08-30T07:59:32.033+03:00,Mercedes-Benz,english
13532,2018-08-31T07:20:36.021+03:00,mercedes Benz,english
38525,2018-09-12T17:20:01.045+03:00,MERCEDES BENZ,english
37996,2018-09-12T10:03:30.003+03:00,Mercedes-Benz,english


### Tuning Tolerance

In [20]:
string = u"Mercedes Benz"

In [21]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=1)

print (index.bucket_size())

95080


In [22]:
s = Simhash(get_features(string))
s_dups = index.get_near_dups(s)
print (index.get_near_dups(s))

news_df.iloc[s_dups]

['23321', '13672', '30313', '25078', '13528', '11667', '13532', '38525', '37996']


,crawled,title,language
23321,2018-09-05T13:25:07.016+03:00,Mercedes Benz,english
13672,2018-08-31T08:50:46.001+03:00,mercedes Benz,english
30313,2018-09-08T12:35:00.000+03:00,Mercedes Benz,english
25078,2018-09-06T05:32:58.011+03:00,mercedes-benz,english
13528,2018-08-31T07:20:15.003+03:00,mercedes Benz,english
11667,2018-08-30T07:59:32.033+03:00,Mercedes-Benz,english
13532,2018-08-31T07:20:36.021+03:00,mercedes Benz,english
38525,2018-09-12T17:20:01.045+03:00,MERCEDES BENZ,english
37996,2018-09-12T10:03:30.003+03:00,Mercedes-Benz,english


In [23]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=3)

print (index.bucket_size())

109932


In [24]:
s = Simhash(get_features(string))
s_dups = index.get_near_dups(s)
print (index.get_near_dups(s))

news_df.iloc[s_dups]

['23321', '30313', '13672', '25078', '13528', '11667', '13532', '38525', '37996']


,crawled,title,language
23321,2018-09-05T13:25:07.016+03:00,Mercedes Benz,english
30313,2018-09-08T12:35:00.000+03:00,Mercedes Benz,english
13672,2018-08-31T08:50:46.001+03:00,mercedes Benz,english
25078,2018-09-06T05:32:58.011+03:00,mercedes-benz,english
13528,2018-08-31T07:20:15.003+03:00,mercedes Benz,english
11667,2018-08-30T07:59:32.033+03:00,Mercedes-Benz,english
13532,2018-08-31T07:20:36.021+03:00,mercedes Benz,english
38525,2018-09-12T17:20:01.045+03:00,MERCEDES BENZ,english
37996,2018-09-12T10:03:30.003+03:00,Mercedes-Benz,english


In [25]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=5)

print (index.bucket_size())

18262


In [26]:
s = Simhash(get_features(string))
s_dups = index.get_near_dups(s)
#print (index.get_near_dups(s))

news_df.iloc[s_dups]

Big bucket found. key:bb:0, len:503
Big bucket found. key:194:2, len:440
Big bucket found. key:0:3, len:602
Big bucket found. key:25:4, len:411


,crawled,title,language
23321,2018-09-05T13:25:07.016+03:00,Mercedes Benz,english
30313,2018-09-08T12:35:00.000+03:00,Mercedes Benz,english
13672,2018-08-31T08:50:46.001+03:00,mercedes Benz,english
25078,2018-09-06T05:32:58.011+03:00,mercedes-benz,english
24074,2018-09-05T20:33:50.001+03:00,The Mercedes-Benz EQC,english
13528,2018-08-31T07:20:15.003+03:00,mercedes Benz,english
11667,2018-08-30T07:59:32.033+03:00,Mercedes-Benz,english
44816,2018-09-15T21:24:39.002+03:00,Mercedes-Benz 508D,english
42508,2018-09-14T16:48:58.000+03:00,Mercedes Benz $4300,english
13532,2018-08-31T07:20:36.021+03:00,mercedes Benz,english


In [27]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=10)

print (index.bucket_size())

13462


In [28]:
s = Simhash(get_features(string))
s_dups = index.get_near_dups(s)
#print (index.get_near_dups(s))

news_df.iloc[s_dups]

Big bucket found. key:1b:0, len:5529
Big bucket found. key:5:1, len:3519
Big bucket found. key:5:2, len:2926
Big bucket found. key:1:3, len:2743
Big bucket found. key:14:4, len:4022
Big bucket found. key:c:5, len:4257
Big bucket found. key:0:6, len:3654
Big bucket found. key:0:7, len:5019
Big bucket found. key:5:8, len:4144
Big bucket found. key:1:9, len:3146


,crawled,title,language
25108,2018-09-06T05:57:29.032+03:00,2019 Mercedes Benz EQC First Electric Mercedes,english
5087,2018-08-26T16:18:41.080+03:00,*2012* *Mercedes-Benz* *GLK 350* * 4MATIC AWD 4dr SUV* (_Mercedes-Benz_ _GLK 350_ _SUV_) $16988,english
39160,2018-09-13T00:04:00.000+03:00,Mercedes-Benz GLE.,english
25078,2018-09-06T05:32:58.011+03:00,mercedes-benz,english
13434,2018-08-31T06:00:09.006+03:00,Mercedes-Benz Select by Mercedes-Benz,english
22833,2018-09-05T08:06:53.004+03:00,2003 Mercedes Benz $3500,english
42508,2018-09-14T16:48:58.000+03:00,Mercedes Benz $4300,english
3674,2018-08-25T20:19:09.019+03:00,2000 Mercedes Benz $2500,english
2064,2018-08-24T23:52:48.022+03:00,Mercedes Benz S430 $1300,english
28438,2018-09-07T17:58:42.019+03:00,2016 Mercedes-Benz -,english


In [29]:
objs = [(str(k), Simhash(get_features(v))) for k, v in data.items()]
index = SimhashIndex(objs, k=20)

print (index.bucket_size())

176


In [30]:
s = Simhash(get_features(string))
#s_dups = index.get_near_dups(s)
#print (index.get_near_dups(s))

news_df.iloc[s_dups]

,crawled,title,language
25108,2018-09-06T05:57:29.032+03:00,2019 Mercedes Benz EQC First Electric Mercedes,english
5087,2018-08-26T16:18:41.080+03:00,*2012* *Mercedes-Benz* *GLK 350* * 4MATIC AWD 4dr SUV* (_Mercedes-Benz_ _GLK 350_ _SUV_) $16988,english
39160,2018-09-13T00:04:00.000+03:00,Mercedes-Benz GLE.,english
25078,2018-09-06T05:32:58.011+03:00,mercedes-benz,english
13434,2018-08-31T06:00:09.006+03:00,Mercedes-Benz Select by Mercedes-Benz,english
22833,2018-09-05T08:06:53.004+03:00,2003 Mercedes Benz $3500,english
42508,2018-09-14T16:48:58.000+03:00,Mercedes Benz $4300,english
3674,2018-08-25T20:19:09.019+03:00,2000 Mercedes Benz $2500,english
2064,2018-08-24T23:52:48.022+03:00,Mercedes Benz S430 $1300,english
28438,2018-09-07T17:58:42.019+03:00,2016 Mercedes-Benz -,english
